#  **Análisis y Predicción de Datos Meteorológicos con Métodos de Interpolación**

### **Integrantes**:  
##### Alejandro Chávez - 32.278.392
##### Cesar Dominguez - 30.640.838
##### José Santana - 31.608.829

---
  
En el presente proyecto estaremos analizando con distintos métodos de Interpolación, las temperaturas de una ciudad, con el fin de modelar e intentar predecir estos datos meteorológicos.

Los metodos de interpolación que emplearemos son:
* Interpolación de Taylor
* Interpolación de Lagrange
* Interpolación de Hermite
* Interpolación Polinómica a Trozos
---  
## **Generación de datos**

Para la generación de datos utilizaremos como referencia la ciudad de Valencia, Carabobo (Ubicada en Venezuela), ciudad la cual mantiene una temperatura dentro de un rango comprendido por el valor de la temperatura mínima (20°c) y el valor de la temperatura máxima (30°c) de la última semana en la ciudad. Por lo tanto utilizaremos una función que pueda comprender entre dichos valores. La función a utilizar será:

**y(x)=5⋅sin(x)+25**

In [91]:
lista = []
for x in 1:7
    push!(lista,5 * sin(x) + 25)
end
println(lista)

Any[29.207354924039482, 29.54648713412841, 25.705600040299338, 21.21598752346036, 20.205378626684308, 23.60292250900537, 28.284932993593944]


A continuación se graficará los valores de temperaturas obtenidos en 7 días

In [ ]:
#Instalar paquetes necesarios (Solo hacer 1 vez, puede tardar más o menos 1 minutos)
using Pkg
Pkg.add("Plots")
using Plots

In [412]:
x = 1:length(lista)
y = lista
scatter(x, y, xlabel="Día", ylabel="Valor de la temperatura", title="Temperatura a través de los días", legend=false, marker=:circle)

---

## **Interpolación de Taylor**

La interpolación de Taylor es una técnica matemática que usa la serie de Taylor para aproximar una función mediante un polinomio, basado en la función y sus derivadas en un punto específico.

![Polinomio de Taylor](../../resources/polinomio-taylor.png)

Para la interpolación de Taylor se nos pide escoger un punto especifico para estos datos. Por lo que escogeremos el valor de de f(x=1), es decir escogeremos el valor de la función para el primer día. Para el valor de N escogeremos el mismo del total de días que hemos evaluado, es decir N = 7.

In [ ]:
#Tendremos acceso a la libreria Sympy
using Pkg
Pkg.add("SymPy")
using SymPy

In [414]:
x0 = 1 #Elección del valor del día / Punto especificado = x0
x= symbols("x")
f= 5*sin(x)+25
taylor = f.subs(x,x0)
for k in 1:7
    derivada = diff(f,x)
    taylor = taylor + derivada.subs(x,x0) * ((x-x0)^k)/factorial(k)
    f = derivada
end

print(expand(taylor))



-x^7*cos(1)/1008 - x^6*sin(1)/144 + x^6*cos(1)/144 + x^5*cos(1)/48 + x^5*sin(1)/24 - 25*x^4*cos(1)/144 + 5*x^4*sin(1)/48 - 25*x^3*sin(1)/36 - 65*x^3*cos(1)/144 - 65*x^2*sin(1)/48 + 101*x^2*cos(1)/48 + 389*x*cos(1)/144 + 101*x*sin(1)/24 - 4241*cos(1)/1008 + 389*sin(1)/144 + 25

A continuación, se mostrará la grafica de los datos originales y la interpolación de Taylor.

In [416]:
x = 1:length(lista)
y = lista
scatter(x, y, xlabel="Día", ylabel="Valor de la temperatura", title="Temperatura a través de los días", legend=false, marker=:circle)

In [418]:
plot(taylor,title="Polinomio de taylor",xlims=(x0-2,x0+2),label="Polinomio",xlabel="Día",ylabel="Temperatura")
scatter!([x0], taylor,label="Día especificado")

---

## **Interpolacion de Lagrange**

La interpolación de Lagrange es una de las interpolaciones más útiles en integración numérica, ésta consiste en una representación de polinomios  de la función.

![Polinomio de Langrange](https://misapuntesyacimientos.wordpress.com/wp-content/uploads/2016/05/clip_image003_thumb.png?w=600)

El polinomio de interpolación de Lagrange, simplemente es una reformulación del polinomio de Newton que evita los cálculos de las diferencias divididas.

Ahora con los datos anteriores que utilizamos en la interpolacion de Taylor , vamos a realizar la interpolacion de lagrange
para todos los puntos...

In [ ]:
#Accedemos a libreria SymPy
using Pkg
Pkg.add("SymPy")
using SymPy

In [ ]:
#Datos anteriores
xi = [0, 1, 2, 3, 4, 5]
yi = [29.207354924039482, 29.54648713412841, 25.705600040299338, 21.21598752346036, 20.205378626684308, 23.60292250900537]

In [ ]:
function lagran(x,y)
    n = length(x)
    variable = symbols("x")
    lagra = 0
    for i in 1:n
        nume = 1
        deno = 1
        for j in 1:n
            if j != i 
                nume *= (variable - x[j])
                deno *= (x[i] - x[j])
            end
        end
        lagra += y[i]*nume/deno
    end
    return lagra
end


A continuación se graficará los valores de temperaturas obtenidos en 7 días

In [422]:
x = 1:length(lista)
y = lista
scatter(x, y, xlabel="Día", ylabel="Valor de la temperatura", title="Temperatura a través de los días", legend=false, marker=:circle)

In [424]:
x_inter = 0:6
result = lagran(xi,yi)
y_inter = [result(i) for i in x_inter]
plot(xi,yi,seriestype=:scatter, label="Temperaturas")
plot!(x_inter,y_inter,title="Polinomio de Lagrange",legend=false)

---

## **Interpolacion de Hermite**

Es un método de interpolación de puntos de datos como una función polinómica. El polinomio de Hermite permite relacionar puntos conociendo sus primera derivadas y sus funciones, y trabaja utilizando el metodo de Lagrange.

El polinomio interpolador de hermite viene dado por:  
<img src="../../resources/hermite_def.png" width="50%"><img src="../../resources/hermite_res.png" width="50%">

Una vez conocido la definicion y la construccion del polinomio interpolador de hermite, ahora realizaremos el polinomio en forma de codigo, utilizando el lenguaje de programacion **Julia**.

Primeramente necesitaremos la libreria Plots, la cual fue previamente importada: 

In [30]:
using Plots

Y declararemos los datos:

In [ ]:
x = [0, 1, 2, 3, 4, 5]
y = [29.207354924039482, 29.54648713412841, 25.705600040299338, 21.21598752346036, 20.205378626684308, 23.60292250900537]
dy_dx = [4.48, 4.37, 3.98, 3.27, 3.10, 3.67]

**x** : Siendo los dias de la semana  
**y** : Siendo sus temperaturas evaluadas en la funcion:  
**y(x)=5⋅sin(x)+25**  
**dy_dx** = Siendo la derivada de la funcion siendo evaluada en las temperaturas

Crearemos una funcion que retorne el metodo de hermite:

In [ ]:
function interpolacion_hermite(x, y, dy_dx, x_interp) #x_interp : Valor para interpolar
    n = length(x)
    resultado = 0
    for i in 1:n
        li = 1
        for j in 1:n
            if i != j
                #Se utiliza el metodo de Lagrange por cada valor utilizando productoria
                li *= (x_interp - x[j]) / (x[i] - x[j])
            end
        end
        hi = (1 - 2 * (x_interp - x[i]) * sum(1 / (x[i] - x[j]) for j in 1:n if i != j)) * li^2
        hj = (x_interp - x[i]) * li^2
        #Se hace la sumatoria total
        resultado += y[i] * hi + dy_dx[i] * hj
    end
    return resultado
end

Y desplegamos dicha funcion utilizando la herramienta grafica Plots:

In [430]:
x_interp = range(0, 5, length=100)
#Interpolamos cada valor
y_interp = [interpolacion_hermite(x, y, dy_dx, xi) for xi in x_interp]

# Creando Grafico
plot(x, y, seriestype=:scatter, label="Temperaturas")
plot!(x_interp, y_interp, label="Interpolacion de Hermite")
xlabel!("x")
ylabel!("y")
title!("Metodo de Interpolacion de Hermite")

## **Interpolación Polinómica a Trozos**

La interpolacion Polinomica a trozos su  idea es subdividir el intervalo [a, b] en
subintervalos, e interpolar en cada subintervalo por un polinomio de grado fijo, de
modo que la funcion interpolante sea globalmente continua. Esto aumentara la precision de la interpolacion

In [ ]:
#Datos:
x = [0, 1, 2, 3, 4, 5]
y = [29.207354924039482, 29.54648713412841, 25.705600040299338, 21.21598752346036, 20.205378626684308, 23.60292250900537]

**x** : Siendo los dias de la semana  
**y** : Siendo sus temperaturas evaluadas en la funcion:  
**y(x)=5⋅sin(x)+25**  

Importamos las librerias necesarias: 

In [293]:
using Plots

Ahora se realiza la interpolacion de lagrange previamente planteada pero agregando un parametro para cada valor del intervalo

In [403]:
using Plots

x = [0, 1, 2, 3, 4, 5]
y = [29.207354924039482, 29.54648713412841, 25.705600040299338, 21.21598752346036, 20.205378626684308, 23.60292250900537]

#x_interp valor a interpolar
function inter_lagran(x, y, x_interp)
    n = length(x)
    lagra = 0
    for i in 1:n
        nume = 1
        deno = 1
        for j in 1:n
            if j != i
                nume *= (x_interp - x[j])
                deno *= (x[i] - x[j])
            end
        end
        lagra += y[i] * nume / deno
    end
    return lagra
end

#Length la cantidad de punto o intervalos que tendra
x_plot = range(0, stop=5, length=30)
y_plot = [inter_lagran(x, y, xi) for xi in x_plot]

plot(x, y, seriestype=:scatter, label="Temperaturas")
plot!(x_plot, y_plot,seriestype=:scatter, label="Lagrange", xlabel="x", ylabel="y", title="Interpolada con Lagrange a Trozos")